## Pipeline Definition

In [1]:
from kfp import compiler
from kfp.dsl import component, pipeline

## components
@component
def preprocess(data: str) -> int:
    return(int(data))

@component
def training(data: int) -> int:
    return(data*234545632)

@component
def deploy(data: int) -> str:
    return(str(data))

## pipelines
@pipeline(name="dummy-pipe")
def pipeline():
    pre_job = preprocess(data="8")
    train_job = training(data=pre_job.output)
    dep_job = deploy(data=train_job.output)

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path='dummy.yaml')

## [Optional] Create Template

In [2]:
from kfp.registry import RegistryClient

client = RegistryClient(host=f"https://us-central1-kfp.pkg.dev/vtxdemos/kfp-repo")

templateName, versionName = client.upload_pipeline(
  file_name="dummy.yaml",
  tags=["v1", "latest"]
  )

## Pipeline Job

In [3]:
from google.cloud import aiplatform

job = aiplatform.PipelineJob(
    display_name="dummy-pipe",
    template_path="dummy.yaml",
    parameter_values={},
    )

job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/254356041555/locations/us-central1/pipelineJobs/dummy-pipe-20230525121512
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/254356041555/locations/us-central1/pipelineJobs/dummy-pipe-20230525121512')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/dummy-pipe-20230525121512?project=254356041555
